In [2]:
import pandas as pd
from pathlib import Path
base_output = Path('group_results')

group_res = {}
group_f = {}

for fp in base_output.rglob('group_stats.csv'):
    group_res[fp.parent.stem] = pd.read_csv(
        fp,
        index_col=0,
        header=[0])
    group_f[fp.parent.stem] = pd.read_csv(
        fp.parent / 'group_fstats.csv',
        index_col=0,
        header=[0])
    
combined_res = {key: pd.concat((group_res[key].loc[:, ['COPE', 'VARCOPE', 'z']] , group_f[key]), axis=1) for key in group_res}


def print_percentage_activations(df):
    mean_activation_df = df.filter(regex='mean_activation', axis=0)
    mean_activation_df.index = mean_activation_df.index.str.replace('_mean_activation','').str.replace('conc_','')
    mean_activation_df = mean_activation_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
    mean_activation_df

    constant_df = df.filter(regex='constant', axis=0)
    constant_df.index = constant_df.index.str.replace('_constant','').str.replace('conc_','')
    constant_df = constant_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
    constant_df.COPE

    mean_activation_df
    # constant_df
    mean_activation_df.loc[:, ['COPE', 'VARCOPE']] = mean_activation_df.loc[:, ['COPE', 'VARCOPE']].div(constant_df.COPE, axis=0)

    return mean_activation_df.style\
    .format(formatter={
        'COPE': "{:+0.1%}",
        'VARCOPE': "{:+0.2%}",
        'z': "{:+05.2f}",
        'p': "{:05.3f}"})\
    .highlight_between(subset=['p'],left=0, right=0.05, props='font-weight:bold;color:#e83e8c')


In [14]:
def print_percentage_activations_all(all_df):
    out = {}
    for key, df in all_df.items():
        mean_activation_df = df.filter(regex='mean_activation', axis=0)
        mean_activation_df.index = mean_activation_df.index.str.replace('_mean_activation','').str.replace('conc_','')
        mean_activation_df = mean_activation_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
        mean_activation_df

        constant_df = df.filter(regex='constant', axis=0)
        constant_df.index = constant_df.index.str.replace('_constant','').str.replace('conc_','')
        constant_df = constant_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
        constant_df.COPE

        mean_activation_df
        # constant_df
        mean_activation_df.loc[:, ['COPE', 'VARCOPE']] = mean_activation_df.loc[:, ['COPE', 'VARCOPE']].div(constant_df.COPE, axis=0)
        out[key] = mean_activation_df.loc[:, ['COPE', 'z', 'f-stat', 'zf-stat','p']]
    
    out = pd.concat(out.values(), axis=1, keys=out.keys())

    formatter_dict = {}
    p_list = []
    for key in all_df:
        formatter_dict.update({
            (key, 'COPE'): "{:+0.1%}",
            #(key, 'VARCOPE'): "{:+0.2%}",
            (key, 'z'): "{:+0.2f}",
            (key, 'f-stat'): "{:+0.2f}",
            (key, 'zf-stat'): "{:+0.2f}",
            (key, 'p'): "{:0.1e}"})
        p_list.append((key, 'p'))
    return out.style\
    .format(formatter=formatter_dict)\
    .highlight_between(subset=p_list,left=0, right=0.05, props='font-weight:bold;color:#e83e8c')

print_percentage_activations_all(combined_res)

In [12]:
import numpy as np
group_res['7T_exact_lb'].filter(regex=r'gamma', axis=0).loc[:,['COPE', 'VARCOPE']].div(np.pi)

,COPE,VARCOPE
gamma_0_bold_lb,0.935708,0.000066


In [6]:
group_res['3T_exact_lb'].filter(regex=r'gamma', axis=0).loc[:,['COPE', 'VARCOPE']].div(np.pi)

,COPE,VARCOPE
gamma_0_bold_lb,1.254033,0.000331


In [7]:
print_percentage_activations(group_res['3T_exact_lb'])

,COPE,VARCOPE,z,p
Ala,-80.6%,+0.01%,-1.70,0.956
Asc,-0.8%,+0.00%,-1.38,0.916
Asp,+2.2%,+0.00%,+7.67,0.000
GABA,+6.9%,+0.00%,+9.20,0.000
GSH,-0.3%,+0.00%,-0.84,0.799
Glc,+2.6%,+0.00%,+3.25,0.001
Gln,+0.2%,+0.00%,+1.06,0.145
Glu,-0.6%,+0.00%,-6.81,1.000
Gly,-51.8%,+0.00%,-2.72,0.997
Ins,-0.2%,+0.00%,-2.38,0.991


In [8]:
print_percentage_activations(group_res['7T_fixed_lb'])

,COPE,VARCOPE,z,p
Ala,-102.0%,+0.00%,-1.79,0.964
Asc,+5.6%,+0.00%,+13.53,0.000
Asp,-1.2%,+0.00%,-7.13,1.000
GABA,-18.9%,+0.00%,-21.34,1.000
GSH,+1.7%,+0.00%,+10.37,0.000
Glc,-19.1%,+0.00%,-19.62,1.000
Gln,-1.0%,+0.00%,-8.65,1.000
Glu,+2.5%,+0.00%,+22.27,0.000
Gly,-inf%,+inf%,-16.49,1.000
Ins,+2.0%,+0.00%,+21.22,0.000


In [9]:
print_percentage_activations(group_res['7T_exact_lb'])

,COPE,VARCOPE,z,p
Ala,-128.0%,+0.00%,-2.86,0.998
Asc,-0.2%,+0.00%,-0.63,0.735
Asp,+0.6%,+0.00%,+3.84,0.000
GABA,-0.4%,+0.00%,-1.15,0.875
GSH,+0.2%,+0.00%,+1.37,0.086
Glc,-0.8%,+0.00%,-1.90,0.971
Gln,-0.1%,+0.00%,-1.15,0.876
Glu,+0.2%,+0.00%,+6.21,0.000
Gly,-54574.1%,+0.09%,-15.69,1.000
Ins,+0.6%,+0.00%,+13.25,0.000


In [10]:
group_res['7T_exact_lb'].filter(regex='gamma', axis=0)

,COPE,VARCOPE,z,p
gamma_0_bold_lb,2.939614,0.000208,27.097242,5.305721e-162


In [11]:
group_res['3T_exact_lb'].filter(regex='gamma', axis=0)

,COPE,VARCOPE,z,p
gamma_0_bold_lb,3.93966,0.001041,24.592104,7.672199e-134


## Cohen's d

In [25]:
to_drop = ['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC', 'Ala', 'Gly', 'Lip09', 'Lip13a', 'Lip13b', 'Lip20']

def calc_cohens_d(all_df):
    out = {}
    for key, df in all_df.items():
        mean_activation_df = df.filter(regex='mean_activation', axis=0)
        mean_activation_df.index = mean_activation_df.index.str.replace('_mean_activation','').str.replace('conc_','')
        mean_activation_df = mean_activation_df.drop(to_drop)
        
        # calculate SD from variance
        mean_activation_df.loc[:,'SDCOPE'] = mean_activation_df.VARCOPE.pow(0.5)

        # Calculate Cohens d
        mean_activation_df.loc[:,'Cohens_d'] = mean_activation_df.COPE.div(mean_activation_df.SDCOPE)

        constant_df = df.filter(regex='constant', axis=0)
        constant_df.index = constant_df.index.str.replace('_constant','').str.replace('conc_','')
        constant_df = constant_df.drop(to_drop)

        mean_activation_df.loc[:, ['COPE', 'VARCOPE']] = mean_activation_df.loc[:, ['COPE', 'VARCOPE', 'SDCOPE']].div(constant_df.COPE, axis=0)
        out[key] = mean_activation_df.loc[:, ['COPE', 'SDCOPE', 'Cohens_d']]
    
    return pd.concat(out.values(), axis=1, keys=out.keys())

def print_cohensd_all(all_df):
    formatter_dict = {}
    for key in results_df.columns.get_level_values(0).unique():
        formatter_dict.update({
            (key, 'COPE'): "{:+0.1%}",
            (key, 'SDCOPE'): "{:0.3%}",
            (key, 'Cohens_d'): "{:0.3f}"})

    return all_df.style\
        .format(formatter=formatter_dict)

results_df = calc_cohens_d(combined_res)

print_cohensd_all(results_df)

In [26]:
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

Index(['7T_correct_lb', '3T_correct_lb', '7T_exact_lb', '3T_exact_lb',
       '7T_fixed_lb', '3T_fixed_lb'],
      dtype='object')